In [10]:
import pandas as pd

# Step 2: Load stock_raw.csv with daily price and share info
stock_df = pd.read_csv('/Users/tristanfeng/Desktop/returns_299.csv')



In [12]:

stock_df['market_cap'] = stock_df['PRC'].abs() * stock_df['SHROUT'] * 1000

# Step 6: Calculate total market cap by day and compute weights
total_mkt_cap = stock_df.groupby('date')['market_cap'].sum().to_frame('total_mkt_cap')
stock_df = stock_df.merge(total_mkt_cap, on='date')
stock_df['mkt_weight'] = stock_df['market_cap'] / stock_df['total_mkt_cap']

# (Optional) Pivot to matrix with dates as rows, permnos as columns (if needed later)
# Fix the pivot issue using pivot_table with aggregation
weight_matrix = stock_df.pivot_table(
    index='date',
    columns='PERMNO',
    values='mkt_weight',
    aggfunc='mean'  # or 'first' depending on your preference
)




In [13]:
stock_df

,PERMNO,date,TICKER,COMNAM,BIDLO,ASKHI,PRC,VOL,RET,BID,...,RETX,vwretd,vwretx,ewretd,ewretx,sprtrn,avg_impl_vol,market_cap,total_mkt_cap,mkt_weight
0,10104,2011-01-03,ORCL,ORACLE CORP,31.52500,31.94000,31.62000,21136353.0,0.010224,31.64000,...,0.010224,0.011212,0.011181,0.012533,0.012516,0.011315,0.218368,1.597575e+11,9.780957e+12,0.016334
1,10104,2011-01-04,ORCL,ORACLE CORP,31.13500,31.75000,31.48000,22978313.0,-0.004428,31.46000,...,-0.004428,-0.003895,-0.003939,-0.006753,-0.006774,-0.001313,0.219083,1.590502e+11,9.765819e+12,0.016286
2,10104,2011-01-05,ORCL,ORACLE CORP,30.98000,31.44000,31.04000,36464087.0,-0.013977,31.03000,...,-0.013977,0.005382,0.005299,0.007944,0.007917,0.005007,0.214034,1.568271e+11,9.816084e+12,0.015977
3,10104,2011-01-06,ORCL,ORACLE CORP,31.02000,31.20000,31.17000,21963429.0,0.004188,31.16000,...,0.004188,-0.002515,-0.002757,-0.001211,-0.001248,-0.002123,0.220563,1.574839e+11,9.793832e+12,0.016080
4,10104,2011-01-07,ORCL,ORACLE CORP,30.93000,31.34000,31.03000,27819266.0,-0.004491,31.03000,...,-0.004491,-0.001949,-0.001951,-0.002848,-0.002850,-0.001845,0.222230,1.567766e+11,9.776637e+12,0.016036
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
902975,92655,2022-12-23,UNH,UNITEDHEALTH GROUP INC,522.89502,531.31000,531.31000,1292327.0,0.008006,531.23999,...,0.008006,0.005465,0.005393,0.002459,0.002195,0.005868,0.232563,4.964290e+11,2.625023e+13,0.018911
902976,92655,2022-12-27,UNH,UNITEDHEALTH GROUP INC,529.84497,535.84003,531.98999,1596719.0,0.001280,531.87000,...,0.001280,-0.003930,-0.003999,-0.008489,-0.008633,-0.004050,0.248637,4.970643e+11,2.618788e+13,0.018981
902977,92655,2022-12-28,UNH,UNITEDHEALTH GROUP INC,527.73401,538.15002,528.45001,1694377.0,-0.006654,528.44000,...,-0.006654,-0.012364,-0.012381,-0.010297,-0.010560,-0.012021,0.245414,4.937567e+11,2.586023e+13,0.019093
902978,92655,2022-12-29,UNH,UNITEDHEALTH GROUP INC,528.85999,533.67999,529.88000,1379681.0,0.002706,529.84003,...,0.002706,0.018339,0.018137,0.022834,0.022336,0.017461,0.247116,4.950928e+11,2.626473e+13,0.018850


In [33]:
import pandas as pd
import numpy as np
import yfinance as yf


# Step 2: Download VIX and convert to daily variance
vix = yf.download('^VIX', start='2011-01-01', end='2022-12-31')[['Close']]
vix.rename(columns={'Close': 'VIX'}, inplace=True)
vix.dropna(inplace=True)
vix['vix_var'] = (vix['VIX'] / 100) ** 2 / 252

# Step 3: Compute daily implied covariance matrices
cov_matrices = {}

for date, group in stock_df.groupby('date'):
    if date not in vix.index:
        continue

    # Extract weights and implied vol
    # Filter and align
    group = group.set_index('PERMNO')
    group = group[['mkt_weight', 'avg_impl_vol']].dropna()

    if len(group) < 2:
        continue

    permnos = group.index.to_list()
    sigma_arr = group['avg_impl_vol'].to_numpy(copy=True)
    w_arr = group['mkt_weight'].to_numpy(copy=True)


    # Market implied variance from VIX
    sigma_mkt = vix.loc[date, 'vix_var'].item()
    top = sigma_mkt - np.sum((w_arr ** 2) * (sigma_arr ** 2))

    # Denominator: off-diagonal weighted vol product
    w_outer = np.outer(w_arr, w_arr)
    sigma_outer = np.outer(sigma_arr, sigma_arr)
    np.fill_diagonal(w_outer, 0)
    np.fill_diagonal(sigma_outer, 0)
    bottom = np.sum(w_outer * sigma_outer)

    # Average implied correlation
    rho_bar = top / bottom if bottom != 0 else 0

    # Covariance matrix
    cov = rho_bar * np.outer(sigma_arr, sigma_arr)
    np.fill_diagonal(cov, np.square(sigma_arr))

    # Store with proper index
    # Store with proper index
    #   (Note: permnos is already defined above correctly)
    cov_df = pd.DataFrame(cov, index=permnos, columns=permnos)

    cov_df = pd.DataFrame(cov, index=permnos, columns=permnos)
    cov_matrices[date] = cov_df

# Step 4: Combine into panel
cov_panel = pd.concat(cov_matrices, axis=0)
cov_panel.index.names = ['date', 'permno_i']

# Step 5: Display
print(cov_panel.head(10))

cov_panel.to_csv("daily_covariance_matrix.csv")


[*********************100%***********************]  1 of 1 completed


                        10104     10107     10138     10145     10516  \
date       permno_i                                                     
2011-01-03 10104     0.047685 -0.000404 -0.000469 -0.000379 -0.000488   
           10107    -0.000404  0.055446 -0.000506 -0.000408 -0.000527   
           10138    -0.000469 -0.000506  0.074765 -0.000474 -0.000612   
           10145    -0.000379 -0.000408 -0.000474  0.048716 -0.000494   
           10516    -0.000488 -0.000527 -0.000612 -0.000494  0.081007   
           11308    -0.000259 -0.000280 -0.000325 -0.000262 -0.000338   
           11404    -0.000203 -0.000219 -0.000254 -0.000205 -0.000264   
           11600    -0.000382 -0.000412 -0.000479 -0.000387 -0.000499   
           11618    -0.000505 -0.000544 -0.000632 -0.000510 -0.000658   
           11674    -0.000272 -0.000293 -0.000341 -0.000275 -0.000355   

                        11308     11404     11600     11618     11674  ...  \
date       permno_i                          

Portfolio With Constraints

In [50]:
import cvxpy as cp

minvar_returns = []
minvar_weights = []

for date, cov_df in cov_panel.groupby(level=0):
    cov_matrix = cov_df.droplevel(0)
    assets = cov_matrix.columns

    # Get valid returns
    day_returns = returns_df.loc[date, assets].dropna()
    valid_assets = day_returns.index
    cov_matrix = cov_matrix.loc[valid_assets, valid_assets]

    if len(valid_assets) < 2:
        minvar_returns.append(np.nan)
        minvar_weights.append(np.nan)
        continue

    # Setup optimization
    N = len(valid_assets)
    w = cp.Variable(N)
    # Symmetrize and ensure PSD by clipping small negative eigenvalues
    Sigma = cov_matrix.values
    eigvals, eigvecs = np.linalg.eigh(Sigma)
    eigvals_clipped = np.clip(eigvals, a_min=0, a_max=None)
    Sigma_psd = eigvecs @ np.diag(eigvals_clipped) @ eigvecs.T

    Sigma_param = cp.Parameter((N, N), PSD=True)
    Sigma_param.value = Sigma_psd
    objective = cp.Minimize(cp.quad_form(w, Sigma_param))

    constraints = [cp.sum(w) == 1, w >= 0]  # no short-selling
    prob = cp.Problem(objective, constraints)

    try:
        prob.solve(solver=cp.SCS)
        weights = w.value
        port_return = weights @ day_returns.loc[valid_assets].values
        minvar_returns.append(port_return)
        minvar_weights.append(pd.Series(weights, index=valid_assets, name=date))
    except Exception as e:
        print(f"Optimization failed on {date}: {e}")
        minvar_returns.append(np.nan)
        minvar_weights.append(np.nan)



/opt/anaconda3/lib/python3.12/site-packages/cvxpy/reductions/solvers/solving_chain.py:241: UserWarning: You are solving a parameterized problem that is not DPP. Because the problem is not DPP, subsequent solves will not be faster than the first one. For more information, see the documentation on Disciplined Parametrized Programming, at https://www.cvxpy.org/tutorial/dpp/index.html
  warnings.warn(DPP_ERROR_MSG)


In [51]:
minvar_returns

[0.0113373195361616,
 -0.0050824700443238375,
 0.006107803934031062,
 -0.0022318958880386884,
 -0.0010300486058797509,
 -0.0005722796548058463,
 0.004594353687197928,
 0.009057609513054137,
 -0.000862056976956793,
 0.00798764328178211,
 0.0026549110256263602,
 -0.012094862050081095,
 -0.001118027836304351,
 0.0019489045467979053,
 0.0051230761182756955,
 0.0006069914553899289,
 0.006033404893372196,
 0.0052339365237030425,
 -0.017854978468046782,
 0.006905936614814197,
 0.014947404499166171,
 -0.002528779962246983,
 0.0036074272931355626,
 0.004612525483878659,
 0.005628439361811019,
 0.00465668738290376,
 -0.00215927414938699,
 0.001702418415049971,
 0.006119767919801243,
 0.00027109020360996486,
 -0.0026331902946635207,
 0.006158168862594947,
 0.0037133293364103874,
 0.002756660825086012,
 -0.023343095020603395,
 -0.008951967806166949,
 -4.273373438527271e-05,
 0.012790173220323137,
 0.0048196061728030554,
 -0.016631570614585933,
 0.0019231342214452195,
 0.018318306693810613,
 -0.006

In [55]:
# Ensure returns are a clean pandas Series
minvar_returns = pd.Series(minvar_returns).dropna()

# Cumulative return
cumulative_return = (1 + minvar_returns).prod() - 1

# Annual volatility
annual_volatility = minvar_returns.std() * np.sqrt(252)

# Sharpe ratio (assuming risk-free rate = 0)
sharpe_ratio = minvar_returns.mean() / minvar_returns.std() * np.sqrt(252)

# Output
print("Minimum-Variance Portfolio Performance:")
print("Cumulative Return:", cumulative_return)
print("Annual Volatility:", annual_volatility)
print("Sharpe Ratio:", sharpe_ratio)


Minimum-Variance Portfolio Performance:
Cumulative Return: 3.5441905962271587
Annual Volatility: 0.1887025665830559
Sharpe Ratio: 0.7643705732573306


Portfolio Without Constraints

In [38]:
import pandas as pd
import numpy as np

# Assuming you already have:
# cov_panel: DataFrame with MultiIndex ['date', 'permno_i'] and columns as permno_j
# stock_df: contains columns ['date', 'PERMNO', 'RET'] for returns

# Step 1: Pivot daily returns into [date x PERMNO] matrix
returns_df = stock_df.pivot(index='date', columns='PERMNO', values='RET')

# Step 2: Equal-weighted portfolio
equal_weights = returns_df.notna().astype(float).div(returns_df.notna().sum(axis=1), axis=0)
equal_returns = (returns_df * equal_weights).sum(axis=1)

# Step 3: Minimum-variance portfolio
minvar_returns = []
minvar_weights = []

for date, cov_df in cov_panel.groupby(level=0):
    cov_matrix = cov_df.droplevel(0)
    assets = cov_matrix.columns

    # Filter returns on the same day to match assets in cov matrix
    day_returns = returns_df.loc[date, assets].dropna()
    cov_matrix = cov_matrix.loc[day_returns.index, day_returns.index]

    if cov_matrix.shape[0] < 2:
        minvar_returns.append(np.nan)
        minvar_weights.append(np.nan)
        continue

    Sigma_inv = np.linalg.pinv(cov_matrix.values)
    e = np.ones(Sigma_inv.shape[0])
    weights = Sigma_inv @ e / (e.T @ Sigma_inv @ e)

    # Save portfolio return and weights
    port_return = weights @ day_returns.loc[day_returns.index].values
    minvar_returns.append(port_return)
    minvar_weights.append(pd.Series(weights, index=day_returns.index, name=date))

# Combine
minvar_returns = pd.Series(minvar_returns, index=cov_panel.index.levels[0], name='minvar')
equal_returns.name = 'equal_weighted'

# Combine into result
returns_combined = pd.concat([equal_returns, minvar_returns], axis=1)
print(returns_combined)


            equal_weighted    minvar
date                                
2011-01-03        0.011146  0.011919
2011-01-04       -0.004924 -0.005617
2011-01-05        0.006004  0.006573
2011-01-06       -0.002146 -0.002340
2011-01-07       -0.000961 -0.001183
...                    ...       ...
2022-12-23        0.008030  0.009974
2022-12-27        0.002232 -0.000292
2022-12-28       -0.013869 -0.020226
2022-12-29        0.015366  0.025877
2022-12-30       -0.003749 -0.000721

[3020 rows x 2 columns]


In [53]:
equal_ret = returns_combined['equal_weighted']
minvar_ret = returns_combined['minvar']

def compute_metrics(ret_series):
    mean_daily = ret_series.mean()
    std_daily = ret_series.std()
    cum_return = (1 + ret_series).prod() - 1
    ann_return = mean_daily * 252
    ann_vol = std_daily * np.sqrt(252)
    sharpe = ann_return / ann_vol
    return cum_return, ann_vol, sharpe

# Compute metrics for both portfolios
eq_cum, eq_vol, eq_sharpe = compute_metrics(equal_ret)
mv_cum, mv_vol, mv_sharpe = compute_metrics(minvar_ret)

# Build comparison DataFrame
comparison_df = pd.DataFrame({
    'Cumulative Return': [eq_cum, mv_cum],
    'Annual Volatility': [eq_vol, mv_vol],
    'Sharpe Ratio': [eq_sharpe, mv_sharpe]
}, index=['Equal Weighted', 'Min-Var'])

print(comparison_df)

                Cumulative Return  Annual Volatility  Sharpe Ratio
Equal Weighted           4.017053           0.184415      0.822582
Min-Var                  1.043693           0.224286      0.378478
